In [8]:
'''
import necessary packages and functions
'''

import os
import re
import io
import csv

from skimage import io, color
from skimage.color import rgb2gray
from skimage.io import imsave
from skimage.transform import rescale, resize, downscale_local_mean


In [7]:
'''
import images from directory, convert to grayscale, append to list
'''
image_dir = '../../dataset/images/'
images = []
image_names = []

def import_images(dir_name, images, image_names):
    for root, dirnames, filenames in os.walk(dir_name):
        for filename in filenames:
            if re.search("\.(jpg|jpeg|png|bmp|tiff)$", filename):
                filepath = os.path.join(root, filename)
                image_names.append(filename)
                
                ## read images and add to list
                image = io.imread(fname=filepath)
                image_gray = rgb2gray(image)
                images.append(image_gray)
                
import_images(image_dir, images, image_names)
            

In [10]:
'''
move images to 0/1 folders based on labels
'''

SOURCE_ROOT = '../../dataset/images/'
DEST_ROOT = '../../dataset/'

with open('../../dataset/image-labels.csv') as infile:
    next(infile)  # Skip the header row
    reader = csv.reader(infile)
    seen = set()
    for Order, External_ID, Label in reader:
        src = os.path.join(SOURCE_ROOT, External_ID)
        dest = os.path.join(DEST_ROOT, Label, External_ID)
        try:
            os.rename(src, dest)
        except WindowsError as e:
            print (e)
            

In [18]:
'''
setup training, testing, and validation split
'''

import random

yes_dir = '../../dataset/1'
no_dir = '../../dataset/0'
yes_imgs = []
no_imgs = []
random.seed(420)

# create list of .jpg files in yes_img directory
for root, dirs, files in os.walk(yes_dir):
    for file in files:
        if file.endswith('.jpg'):
            yes_imgs.append(file)

# create list of .jpg files in no_img directory
for root, dirs, files in os.walk(no_dir):
    for file in files:
        if file.endswith('.jpg'):
            no_imgs.append(file)            
            
yes_imgs.sort()  # make sure that the filenames have a fixed order before shuffling
random.shuffle(yes_imgs) # shuffles the ordering of filenames (deterministic given the chosen seed)

yes_split_1 = int(0.8 * len(yes_imgs))
yes_split_2 = int(0.9 * len(yes_imgs))
yes_train = yes_imgs[:yes_split_1]
yes_validation = yes_imgs[yes_split_1:yes_split_2]
yes_test = yes_imgs[yes_split_2:]

no_imgs.sort()  # make sure that the filenames have a fixed order before shuffling
random.shuffle(no_imgs) # shuffles the ordering of filenames (deterministic given the chosen seed)

no_split_1 = int(0.8 * len(no_imgs))
no_split_2 = int(0.9 * len(no_imgs))
no_train = no_imgs[:no_split_1]
no_validation = no_imgs[no_split_1:no_split_2]
no_test = no_imgs[no_split_2:]


In [19]:
'''
move images to appropriate folder for training, testing, and validation
'''

def move_images(img_list, src_dir, dest_dir):
    for img in img_list:
        src = os.path.join(src_dir, img)
        dest = os.path.join(dest_dir, img)
        os.rename(src, dest)

# move yes training data
move_images(yes_train, '../../dataset/1/', '../../data/train/1/')
# move no training data
move_images(no_train, '../../dataset/0/', '../../data/train/0/')

# move testing data
move_images(yes_test, '../../dataset/1/', '../../data/test/1/')
# move no testing data
move_images(no_test, '../../dataset/0/', '../../data/test/0/')

# move validation data
move_images(yes_validation, '../../dataset/1/', '../../data/validation/1/')
# move no validation data
move_images(no_validation, '../../dataset/0/', '../../data/validation/0/')

    

In [ ]:
'''
amplify training dataset via image augmentation
'''

